In [8]:
states = {1:"Лифт в покое", 2: "Лифт на месте", 3: "Лифт поднимается", 4:"Лифт опускается"}

In [9]:
class elevators:
  def __init__(self, start_position, number):
    # инициализация
    self.number = number
    self.current_position = start_position
    self.stack_positions = []
    self.next_position = None
    self.error = False
    self.stopped = False
        
  def open_close(self):
    print(f"Лифт {self.number} открыл и закрыл двери на {self.current_position} этаже")
  
  def up(self):
    self.current_position += 1
    print(f"Лифт {self.number} поднялся на {self.current_position} этаж")

  def down(self):
    self.current_position -= 1
    print(f"Лифт {self.number} опустился на {self.current_position} этаж")

  def state1(self):
    self.next_position = self.stack_positions.pop(0)
      

  def state2(self):
    self.open_close()

  def state3(self):
    while self.current_position < self.next_position:
      self.up()
    self.state2()

  def state4(self):
    while self.current_position > self.next_position:
      self.down()
    self.state2()

  def run(self, stack_positions):
    self.stack_positions = stack_positions
    while(len(self.stack_positions) > 0):
      self.state1()
      if self.current_position == self.next_position:
        self.state = 2
        self.state2()
      elif self.current_position < self.next_position:
        self.state = 3
        self.state3()
      else:
        self.state = 4
        self.state4()
    print(f"Лифт {self.number} в покое")

In [10]:
class elevators_controller:
  def __init__(self, start_position1, start_position2, floor_count):
    self.floor_count = floor_count
    self.elevator1 = elevators(start_position1, 1)
    self.elevator2 = elevators(start_position2, 2)
    self.stack = []
    self.stack_elevator1 = []
    self.stack_elevator2 = []
    self.current_command = None
    self.next_command = None
    self.error = False

  def Error(self, command):
    if command[0] > self.floor_count or command[1] > self.floor_count or command[0] < 1 or command[1] < 1:
      print("Ошибка номера этажа не существует")
      self.error = True

  def initialize_stack(self, stack):
    self.stack.extend(stack)

  def check_way(self, command):
    if command[0] < command[1]:
      return "up"
    elif command[0] > command[1]:
      return "down"

  def return_empty(self):
    if len(self.stack_elevator1) == 0 and len(self.stack_elevator2) == 0:
      return "all"
    elif len(self.stack_elevator1) == 0:
      return 1
    elif len(self.stack_elevator2) == 0:
      return 2
    else:
      "None"
    

  def change_nearest(self, command):
    if(len(self.stack_elevator1)>0 and len(self.stack_elevator2)>0):
      if abs(self.stack_elevator1[-1] - command[0]) <= abs(self.stack_elevator2[-1] - command[0]):
        return 1
      else:
        return 2
    else:
      if abs(self.elevator1.current_position - command[0]) <= abs(self.elevator2.current_position - command[0]):
        return 1
      else:
        return 2

  def passenger(self, command, number):
    if number == 1:
      if self.check_way(command) == self.check_way((self.stack_elevator1[-2], self.stack_elevator1[-1])):
        if command[0] >= self.stack_elevator1[-2] and command[0] <= self.stack_elevator1[-1] or command[0] <= self.stack_elevator1[-2] and command[0] >= self.stack_elevator1[-1]:
          self.stack_elevator1.insert(-1, command[0])
          self.stack_elevator1.append(command[1])
          return True
      return False
    elif number == 2:
      if self.check_way(command) == self.check_way((self.stack_elevator2[-2], self.stack_elevator2[-1])):
        if command[0] >= self.stack_elevator2[-2] and command[0] <= self.stack_elevator2[-1] or command[0] <= self.stack_elevator2[-2] and command[0] >= self.stack_elevator2[-1]:
          self.stack_elevator2.insert(-1, command[0])
          self.stack_elevator2.append(command[1])
          return True
      return False

  def optimize(self, stack):
    for i in range(len(stack) - 1, 0, -1):
      if stack[i] == stack[i-1]:
        del stack[i]

    

  def distribution(self):
    while(len(self.stack) > 0):
      isPassenger = False
      self.current_command = self.stack.pop(0)
      self.Error(self.current_command)
      if self.error:
        return
      isEmpty = self.return_empty()
      if(isEmpty == "all"):
        if self.change_nearest(self.current_command) == 1:
          self.stack_elevator1.append(self.current_command[0])
          self.stack_elevator1.append(self.current_command[1])
        else:
          self.stack_elevator2.append(self.current_command[0])
          self.stack_elevator2.append(self.current_command[1])
      elif(isEmpty == 1):
        isPassenger = self.passenger(self.current_command, 2)
        if isPassenger == False:
          self.stack_elevator1.append(self.current_command[0])
          self.stack_elevator1.append(self.current_command[1])
      elif(isEmpty == 2):
        isPassenger = self.passenger(self.current_command, 1)
        if isPassenger == False:
          self.stack_elevator2.append(self.current_command[0])
          self.stack_elevator2.append(self.current_command[1])  
      else:
        isPassenger1 = self.passenger(self.current_command, 2)
        isPassenger2 = self.passenger(self.current_command, 1)
        if isPassenger1 == False and isPassenger2 == False:
          if self.change_nearest(self.current_command) == 1:
            self.stack_elevator1.append(self.current_command[0])
            self.stack_elevator1.append(self.current_command[1])
          else:
            self.stack_elevator2.append(self.current_command[0])
            self.stack_elevator2.append(self.current_command[1])
      
      self.optimize(self.stack_elevator1)
      self.optimize(self.stack_elevator2)

  def run(self):
    if self.error:
      return
    print(self.stack_elevator1)
    print(self.stack_elevator2)
    self.elevator1.run(self.stack_elevator1)
    self.elevator2.run(self.stack_elevator2)


In [11]:
el = elevators_controller(1, 5, 9)
el.initialize_stack([(1, 5), (5, 6), (5, 8), (2, 4), (3,5), (1, 2)])
el.distribution()
el.run()

[1, 5, 6, 8]
[2, 3, 4, 5, 1, 2]
Лифт 1 открыл и закрыл двери на 1 этаже
Лифт 1 поднялся на 2 этаж
Лифт 1 поднялся на 3 этаж
Лифт 1 поднялся на 4 этаж
Лифт 1 поднялся на 5 этаж
Лифт 1 открыл и закрыл двери на 5 этаже
Лифт 1 поднялся на 6 этаж
Лифт 1 открыл и закрыл двери на 6 этаже
Лифт 1 поднялся на 7 этаж
Лифт 1 поднялся на 8 этаж
Лифт 1 открыл и закрыл двери на 8 этаже
Лифт 1 в покое
Лифт 2 опустился на 4 этаж
Лифт 2 опустился на 3 этаж
Лифт 2 опустился на 2 этаж
Лифт 2 открыл и закрыл двери на 2 этаже
Лифт 2 поднялся на 3 этаж
Лифт 2 открыл и закрыл двери на 3 этаже
Лифт 2 поднялся на 4 этаж
Лифт 2 открыл и закрыл двери на 4 этаже
Лифт 2 поднялся на 5 этаж
Лифт 2 открыл и закрыл двери на 5 этаже
Лифт 2 опустился на 4 этаж
Лифт 2 опустился на 3 этаж
Лифт 2 опустился на 2 этаж
Лифт 2 опустился на 1 этаж
Лифт 2 открыл и закрыл двери на 1 этаже
Лифт 2 поднялся на 2 этаж
Лифт 2 открыл и закрыл двери на 2 этаже
Лифт 2 в покое
